## Chap04-2. Deep NN
### 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델 구현
### 신경망 레이어를 여러개로 구성하여 딥러능 구현

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# 1) 학습 데이터 정의

# [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# x_data에 해당하는 labeling 작업
# [기타, 포유류, 조류] one-hot encoding 방식으로 입력
y_data = np.array([
    [1, 0, 0], # 기타
    [0, 1, 0], # 포유류
    [0, 0, 1], # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

In [13]:
# 2) 신경망 모델 구성
# Hidden layer의 Unit 수를 10개로 설정

# Input(X), Output(Y) 설정
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# input layer --> hidden layer 의 Parameter(W1, b1) 설정
#  - W: [입력층(input) 뉴런 수, 은닉층 뉴런 수(hidden layer unit))] -->> [2, 10]으로 설정 
#  - b: 편향(bias)은 hidden layer의 출력 개수로 설정 [10]
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
b1 = tf.Variable(tf.zeros([10]))

# hidden layer --> ouput layer 의 Parameter(W2, b2) 설정
#  - W: [은닉층 뉴런 수, 출력층 뉴런 수] -->> [10, 3]
#  - b: 편향(bias)은 ouput layer의 출력 개수로 설정 [3]
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))
b2 = tf.Variable(tf.zeros([3]))

# 신경망의 Hidden layer에서 가중치 W1와 편향 b1를 적용
# XW + b 한 값에 ReLU 활성화 함수 적용
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# Output을 계산
#  - Hidden layer의 두번째 가중치 W2와 편향 b2를 적용
model = tf.add(tf.matmul(L1, W2), b2)

# 손실함수(Loss function): Cross-Entropy를 이용
#  - https://www.tensorflow.org/api_docs/python/tf/losses/softmax_cross_entropy
#  - tf.losses.softmax_cross_entropy를 이용
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
cost = tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=model)

# 최적화 함수 설정: Adam을 사용
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [14]:
# 3) 신경망 모델 학습

# Tensorflow 세션 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 최적화
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if(step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 0.823992
20 0.63941
30 0.496998
40 0.382711
50 0.291372
60 0.219118
70 0.16395
80 0.122704
90 0.0923712
100 0.0706638


In [15]:
# 4) 결과 확인
# 학습데이터가 적어 정확도는 매 실행마다 다르게 나옴

# tf.argmax를 이용해 예측한 결과 중 가장 큰 값을 가져옴
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값: ', sess.run(prediction, feed_dict={X: x_data}))
print('실제값: ', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: {:.2f}'.format(sess.run(accuracy * 100, feed_dict={X: x_data, Y:y_data})))

예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도: 100.00
